# Jednoduchý klasifikátor obrázkov

Tento notebook vám ukáže, ako klasifikovať obrázky pomocou predtrénovanej neurónovej siete.

**Čo sa naučíte:**
- Ako načítať a používať predtrénovaný model
- Predspracovanie obrázkov
- Vytváranie predpovedí na obrázkoch
- Pochopenie skóre dôveryhodnosti

**Použitie:** Identifikácia objektov na obrázkoch (napríklad "mačka", "pes", "auto" atď.)

---


## Krok 1: Importovanie potrebných knižníc

Poďme si importovať nástroje, ktoré budeme potrebovať. Nemajte obavy, ak zatiaľ nerozumiete všetkým!


In [ ]:
# Core libraries
import numpy as np
from PIL import Image
import requests
from io import BytesIO

# TensorFlow for deep learning
try:
    import tensorflow as tf
    from tensorflow.keras.applications import MobileNetV2
    from tensorflow.keras.applications.mobilenet_v2 import preprocess_input, decode_predictions
    print("✅ TensorFlow loaded successfully!")
    print(f"   Version: {tf.__version__}")
except ImportError:
    print("❌ Please install TensorFlow: pip install tensorflow")

## Krok 2: Načítanie predtrénovaného modelu

Použijeme **MobileNetV2**, neurónovú sieť, ktorá už bola natrénovaná na miliónoch obrázkov.

Toto sa nazýva **Transfer Learning** - použitie modelu, ktorý natrénoval niekto iný!


In [ ]:
print("📦 Loading pre-trained MobileNetV2 model...")
print("   This may take a minute on first run (downloading weights)...")

# Load the model
# include_top=True means we use the classification layer
# weights='imagenet' means it was trained on ImageNet dataset
model = MobileNetV2(weights='imagenet', include_top=True)

print("✅ Model loaded!")
print(f"   The model can recognize 1000 different object categories")

## Krok 3: Pomocné funkcie

Vytvorme funkcie na načítanie a prípravu obrázkov pre náš model.


In [ ]:
def load_image_from_url(url):
    """
    Load an image from a URL.
    
    Args:
        url: Web address of the image
        
    Returns:
        PIL Image object
    """
    response = requests.get(url)
    img = Image.open(BytesIO(response.content))
    return img


def prepare_image(img):
    """
    Prepare an image for the model.
    
    Steps:
    1. Resize to 224x224 (model's expected size)
    2. Convert to array
    3. Add batch dimension
    4. Preprocess for MobileNetV2
    
    Args:
        img: PIL Image
        
    Returns:
        Preprocessed image array
    """
    # Resize to 224x224 pixels
    img = img.resize((224, 224))
    
    # Convert to numpy array
    img_array = np.array(img)
    
    # Add batch dimension (model expects multiple images)
    img_array = np.expand_dims(img_array, axis=0)
    
    # Preprocess for MobileNetV2
    img_array = preprocess_input(img_array)
    
    return img_array


def classify_image(img):
    """
    Classify an image and return top predictions.
    
    Args:
        img: PIL Image
        
    Returns:
        List of (class_name, confidence) tuples
    """
    # Prepare the image
    img_array = prepare_image(img)
    
    # Make prediction
    predictions = model.predict(img_array, verbose=0)
    
    # Decode predictions to human-readable labels
    # top=5 means we get the top 5 most likely classes
    decoded = decode_predictions(predictions, top=5)[0]
    
    # Convert to simpler format
    results = [(label, float(confidence)) for (_, label, confidence) in decoded]
    
    return results


print("✅ Helper functions ready!")

## Krok 4: Testovanie na vzorových obrázkoch

Skúsme klasifikovať niektoré obrázky z internetu!


In [ ]:
# Sample images to classify
# These are from Unsplash (free stock photos)
test_images = [
    {
        "url": "https://images.unsplash.com/photo-1514888286974-6c03e2ca1dba?w=400",
        "description": "A cat"
    },
    {
        "url": "https://images.unsplash.com/photo-1552053831-71594a27632d?w=400",
        "description": "A dog"
    },
    {
        "url": "https://images.unsplash.com/photo-1511919884226-fd3cad34687c?w=400",
        "description": "A car"
    },
]

print(f"🧪 Testing on {len(test_images)} images...")
print("=" * 70)

### Klasifikujte každý obrázok


In [ ]:
for i, img_data in enumerate(test_images, 1):
    print(f"\n📸 Image {i}: {img_data['description']}")
    print("-" * 70)
    
    try:
        # Load image
        img = load_image_from_url(img_data['url'])
        
        # Display image
        display(img.resize((200, 200)))  # Show smaller version
        
        # Classify
        results = classify_image(img)
        
        # Show predictions
        print("\n🎯 Top 5 Predictions:")
        for rank, (label, confidence) in enumerate(results, 1):
            # Create a visual bar
            bar_length = int(confidence * 50)
            bar = "█" * bar_length
            
            print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
        
    except Exception as e:
        print(f"❌ Error: {e}")

print("\n" + "=" * 70)

## Krok 5: Vyskúšajte vlastné obrázky!

Nahraďte URL nižšie akoukoľvek URL obrázka, ktorý chcete klasifikovať.


In [ ]:
# Try your own image!
# Replace this URL with any image URL
custom_image_url = "https://images.unsplash.com/photo-1472491235688-bdc81a63246e?w=400"  # A flower

print("🖼️  Classifying your custom image...")
print("=" * 70)

try:
    # Load and show image
    img = load_image_from_url(custom_image_url)
    display(img.resize((300, 300)))
    
    # Classify
    results = classify_image(img)
    
    # Show results
    print("\n🎯 Top 5 Predictions:")
    print("-" * 70)
    for rank, (label, confidence) in enumerate(results, 1):
        bar_length = int(confidence * 50)
        bar = "█" * bar_length
        print(f"  {rank}. {label:20s} {confidence*100:5.2f}% {bar}")
    
    # Highlight top prediction
    top_label, top_confidence = results[0]
    print("\n" + "=" * 70)
    print(f"\n🏆 Best guess: {top_label} ({top_confidence*100:.2f}% confident)")
    
except Exception as e:
    print(f"❌ Error: {e}")
    print("   Make sure the URL points to a valid image!")

## 💡 Čo sa práve stalo?

1. **Načítali sme predtrénovaný model** - MobileNetV2 bol natrénovaný na miliónoch obrázkov  
2. **Predspracovali sme obrázky** - Zmenili sme ich veľkosť a upravili ich pre model  
3. **Model vykonal predikcie** - Vrátil pravdepodobnosti pre 1000 kategórií objektov  
4. **Dekódovali sme výsledky** - Preložili sme čísla na čitateľné označenia  

### Pochopenie skóre dôveryhodnosti

- **90-100%**: Veľmi isté (takmer určite správne)  
- **70-90%**: Isté (pravdepodobne správne)  
- **50-70%**: Čiastočne isté (môže byť správne)  
- **Pod 50%**: Málo isté (neisté)  

### Prečo môžu byť predikcie nesprávne?

- **Neobvyklý uhol alebo osvetlenie** - Model bol trénovaný na typických fotografiách  
- **Viacero objektov** - Model očakáva jeden hlavný objekt  
- **Zriedkavé objekty** - Model pozná iba 1000 kategórií  
- **Nekvalitný obrázok** - Rozmazané alebo pixelované obrázky sú náročnejšie  


## 🚀 Ďalšie kroky

1. **Vyskúšajte rôzne obrázky:**
   - Nájdite obrázky na [Unsplash](https://unsplash.com)
   - Kliknite pravým tlačidlom → „Kopírovať adresu obrázka“ na získanie URL

2. **Experimentujte:**
   - Čo sa stane s abstraktným umením?
   - Dokáže rozpoznať objekty z rôznych uhlov?
   - Ako si poradí s viacerými objektmi?

3. **Získajte viac informácií:**
   - Preskúmajte [lekcie o počítačovom videní](../lessons/4-ComputerVision/README.md)
   - Naučte sa trénovať vlastný klasifikátor obrázkov
   - Pochopte, ako fungujú CNN (Konvolučné neurónové siete)

---

## 🎉 Gratulujeme!

Práve ste vytvorili klasifikátor obrázkov pomocou špičkovej neurónovej siete!

Táto technika poháňa:
- Google Photos (organizovanie vašich fotografií)
- Autonómne vozidlá (rozpoznávanie objektov)
- Lekársku diagnostiku (analýza röntgenových snímok)
- Kontrolu kvality (detekcia chýb)

Pokračujte v objavovaní a učení! 🚀



---

**Upozornenie**:  
Tento dokument bol preložený pomocou služby AI prekladu [Co-op Translator](https://github.com/Azure/co-op-translator). Hoci sa snažíme o presnosť, upozorňujeme, že automatizované preklady môžu obsahovať chyby alebo nepresnosti. Pôvodný dokument v jeho pôvodnom jazyku by mal byť považovaný za autoritatívny zdroj. Pre kritické informácie sa odporúča profesionálny ľudský preklad. Nenesieme zodpovednosť za akékoľvek nedorozumenia alebo nesprávne interpretácie vyplývajúce z použitia tohto prekladu.
